In [29]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [30]:
# Define paths to the datasets
happy_dir = '/home/nil/code/Nil-Barcons/Happy Sad/data/happy'
sad_dir = '/home/nil/code/Nil-Barcons/Happy Sad/data/sad'

In [31]:
# Combine happy and sad data
X = np.array(happy_X + sad_X)
y = np.array(happy_y + sad_y)

In [32]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [33]:
# Function to load and preprocess images from directory
def load_images_from_dir(directory):
    X = []
    y = []
    for filename in os.listdir(directory):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img = load_img(os.path.join(directory, filename), target_size=(150, 150))
            img_array = img_to_array(img) / 255.0  # Normalize pixel values
            X.append(img_array)
            if directory == happy_dir:
                y.append(0)  # Label 0 for happy
            elif directory == sad_dir:
                y.append(1)  # Label 1 for sad
    return X, y

In [34]:
# Load and preprocess images from directories
happy_X, happy_y = load_images_from_dir(happy_dir)
sad_X, sad_y = load_images_from_dir(sad_dir)

In [35]:
# Split data into train and test sets
happy_X_train, happy_X_test, happy_y_train, happy_y_test = train_test_split(happy_X, happy_y, test_size=0.2, random_state=42)
sad_X_train, sad_X_test, sad_y_train, sad_y_test = train_test_split(sad_X, sad_y, test_size=0.2, random_state=42)


In [36]:
# Combine happy and sad data
X_train = np.array(happy_X_train + sad_X_train)
X_test = np.array(happy_X_test + sad_X_test)
y_train = to_categorical(np.array(happy_y_train + sad_y_train))
y_test = to_categorical(np.array(happy_y_test + sad_y_test))

In [37]:
# Load pre-trained VGG16 model
vgg16_base = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(150, 150, 3))

In [38]:
# Freeze convolutional base
vgg16_base.trainable = False

In [39]:
# Create new model on top of VGG16
model = Sequential([
    vgg16_base,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 classes: happy and sad
])

In [40]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [41]:
# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    validation_data=(X_test, y_test)
)

Epoch 1/20


2024-03-14 19:43:00.338067: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 652320000 exceeds 10% of free system memory.


76/76 [==============================] - 182s 2s/step - loss: 0.7969 - accuracy: 0.6428 - val_loss: 0.5160 - val_accuracy: 0.7421
Epoch 2/20
76/76 [==============================] - 172s 2s/step - loss: 0.4964 - accuracy: 0.7645 - val_loss: 0.4673 - val_accuracy: 0.7719
Epoch 3/20
76/76 [==============================] - 170s 2s/step - loss: 0.4473 - accuracy: 0.7922 - val_loss: 0.4379 - val_accuracy: 0.8017
Epoch 4/20
76/76 [==============================] - 174s 2s/step - loss: 0.3954 - accuracy: 0.8183 - val_loss: 0.4116 - val_accuracy: 0.8248
Epoch 5/20
76/76 [==============================] - 171s 2s/step - loss: 0.3646 - accuracy: 0.8423 - val_loss: 0.4715 - val_accuracy: 0.7884
Epoch 6/20
76/76 [==============================] - 173s 2s/step - loss: 0.3347 - accuracy: 0.8502 - val_loss: 0.4492 - val_accuracy: 0.8000
Epoch 7/20
76/76 [==============================] - 174s 2s/step - loss: 0.3211 - accuracy: 0.8535 - val_loss: 0.4071 - val_accuracy: 0.8165
Epoch 8/20
76/76 [======

In [42]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_datagen.fit(X_train)

In [43]:
# Compile the model
model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

/home/nil/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [44]:
# Train the model with data augmentation
history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=32),
    steps_per_epoch=len(X_train) / 32,
    epochs=30,
    validation_data=(X_test, y_test)
)

Epoch 1/30
75/75 [==============================] - 170s 2s/step - loss: 0.4458 - accuracy: 0.8055 - val_loss: 0.4198 - val_accuracy: 0.8231
Epoch 2/30
75/75 [==============================] - 169s 2s/step - loss: 0.4029 - accuracy: 0.8373 - val_loss: 0.4328 - val_accuracy: 0.8033
Epoch 3/30
75/75 [==============================] - 165s 2s/step - loss: 0.4113 - accuracy: 0.8200 - val_loss: 0.4089 - val_accuracy: 0.8264
Epoch 4/30
75/75 [==============================] - 165s 2s/step - loss: 0.4023 - accuracy: 0.8204 - val_loss: 0.4142 - val_accuracy: 0.8231
Epoch 5/30
75/75 [==============================] - 165s 2s/step - loss: 0.4061 - accuracy: 0.8212 - val_loss: 0.4116 - val_accuracy: 0.8248
Epoch 6/30
75/75 [==============================] - 169s 2s/step - loss: 0.4006 - accuracy: 0.8253 - val_loss: 0.4019 - val_accuracy: 0.8380
Epoch 7/30
75/75 [==============================] - 174s 2s/step - loss: 0.3722 - accuracy: 0.8340 - val_loss: 0.4023 - val_accuracy: 0.8331
Epoch 8/30
75

In [46]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

19/19 [==============================] - 42s 2s/step - loss: 0.3950 - accuracy: 0.8314
Test accuracy: 0.831404983997345
